In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# subset to relevant urls
humanist_urls = ["https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/", "https://humanist.kdl.kcl.ac.uk/Archives/Current/"]
volume_dfs = []
# loop through each url
for url in humanist_urls:
    print(f"Getting volumes from {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all('a')
    # loop through each volume link
    for link in links:
        if link['href'].endswith('.txt'):
            print(f"Getting volume from {url + link['href']}")
            page_soup = BeautifulSoup(requests.get(url + link['href']).text, "html.parser")
            text = page_soup.get_text()
            volume_link = url + link['href']
            dates = link['href'].split('.')[1]
            data_dict = {'volume_text': text, 'volume_link': volume_link, 'volume_dates': dates}
            volume_dfs.append(data_dict)

scraped_humanist_df = pd.DataFrame(volume_dfs)
# Extract the volume number from the dates
scraped_humanist_df['volume_number'] = scraped_humanist_df['volume_dates'].str.extract(r'(\d+)')
# Remove numbers with more than 2 digits
scraped_humanist_df['volume_number'] = scraped_humanist_df['volume_number'].apply(lambda x: np.nan if len(str(x)) > 2 else x)

# Replace nulls with a sequential of volume numbers
scraped_humanist_df['volume_number'] = scraped_humanist_df['volume_number'].fillna(pd.Series(np.arange(1, len(scraped_humanist_df) + 1)))

# Extract the start and end years
scraped_humanist_df[['inferred_start_year', 'inferred_end_year']] = scraped_humanist_df['volume_dates'].str.split('-', expand=True)

# Remove years that are not 4 digits
scraped_humanist_df.inferred_start_year = scraped_humanist_df.inferred_start_year.apply(lambda x: np.nan if len(str(x)) != 4 else x)
scraped_humanist_df.inferred_end_year = scraped_humanist_df.inferred_end_year.apply(lambda x: np.nan if len(str(x)) != 4 else x)

# Ensure the years are numeric
scraped_humanist_df.loc[scraped_humanist_df.inferred_end_year.isnull(), 'inferred_end_year'] = np.nan

# Create an empty dummy variable for the years
start_year_before = None
end_year_before = None

# Loop through dataframe row by row
for index, row in scraped_humanist_df.iterrows():
    # Check that both start and end years are not null
    if (not pd.isnull(row.inferred_start_year)) and (not pd.isnull(row.inferred_end_year)):
        # assign the years to the dummy variables
        start_year_before = row.inferred_start_year
        end_year_before = row.inferred_end_year
        # print the years
        print(start_year_before, end_year_before)
    # Check that if years are null and the dummy variables are not, then update the years in the dataframe
    elif (pd.isnull(row.inferred_start_year) and start_year_before is not None) and (pd.isnull(row.inferred_end_year) and end_year_before is not None):
        # increment the years by 1
        start_year_before = int(start_year_before) + 1
        end_year_before = int(end_year_before) + 1
        # assign the years to the dataframe using the row index to update the original dataframe
        scraped_humanist_df.at[index, 'inferred_start_year'] = start_year_before
        scraped_humanist_df.at[index, 'inferred_end_year'] = end_year_before
        print(start_year_before, end_year_before)

# Save the dataframe to a csv
scraped_humanist_df.to_csv("web_scraped_humanist_listserv_volumes.csv", index=False)

Getting volumes from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1987-1988.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1988-1989.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1989-1990.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1990-1991.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1991-1992.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1992-1993.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1993-1994.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1994-1995.txt
Getting volume from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.1995-1996.txt
Getting volume from https://humanist.kdl.

In [4]:
import pandas as pd
humanist_vols = pd.read_csv('web_scraped_humanist_listserv_volumes.csv')
# Check the data types of our columns
humanist_vols.dtypes

volume_text            object
volume_link            object
volume_dates           object
volume_number           int64
inferred_start_year     int64
inferred_end_year       int64
dtype: object

In [5]:
humanist_vols['humanities_computing_counts'] = humanist_vols['volume_text'].str.count('humanities computing')
humanist_vols['digital_humanities_counts'] = humanist_vols['volume_text'].str.count('digital humanities')
humanist_vols.head()

,volume_text,volume_link,volume_dates,volume_number,inferred_start_year,inferred_end_year,humanities_computing_counts,digital_humanities_counts
0,From: MCCARTY@UTOREPAS\nSubject: \nDate: 12 Ma...,https://humanist.kdl.kcl.ac.uk/Archives/Conver...,1987-1988,1,1987,1988,98,0
1,From: Sebastian Rahtz \nSubject: C++ and Gnu o...,https://humanist.kdl.kcl.ac.uk/Archives/Conver...,1988-1989,2,1988,1989,55,0
2,From: Willard McCarty \nSubject: Happy Birthda...,https://humanist.kdl.kcl.ac.uk/Archives/Conver...,1989-1990,3,1989,1990,107,0
3,From: Elaine Brennan & Allen Renear \nSubject:...,https://humanist.kdl.kcl.ac.uk/Archives/Conver...,1990-1991,4,1990,1991,29,0
4,From: Elaine Brennan & Allen Renear \nSubject:...,https://humanist.kdl.kcl.ac.uk/Archives/Conver...,1991-1992,5,1991,1992,49,0


In [6]:
import altair as alt
# Convert the inferred start year to a datetime object
humanist_vols['inferred_start_year'] = humanist_vols['inferred_start_year'].astype(str) + '-01-01'
humanist_vols['inferred_start_year'] = pd.to_datetime(humanist_vols['inferred_start_year'])
# Subset the data to only include the volume size and inferred start year so that our chart is not huge
chart = alt.Chart(humanist_vols[['volume_size', 'inferred_start_year']]).mark_bar().encode(
    x='inferred_start_year:T',
    y='volume_size',
    tooltip=['inferred_start_year', 'volume_size']
)

KeyError: "['volume_size'] not in index"

In [8]:

humanist_vols[['humanities_computing_counts', 'digital_humanities_counts']].plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [9]:
humanist_vols['lowercase_humanities_computing_counts'] = humanist_vols['volume_text'].str.lower().str.count('humanities computing')
humanist_vols['lowercase_digital_humanities_counts'] = humanist_vols['volume_text'].str.lower().str.count('digital humanities')
# Plot the data
humanist_vols[['lowercase_humanities_computing_counts', 'lowercase_digital_humanities_counts', 'humanities_computing_counts', 'digital_humanities_counts']].plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

#save our texts to a list
documents = subset_humanist_vols.volume_text.tolist()

#Create a vectorizer
vectorizer = TfidfVectorizer(max_df=.7, min_df=1)


NameError: name 'subset_humanist_vols' is not defined

In [2]:
from nltk import word_tokenize
from nltk import FreqDist

tokens = FreqDist(sum(humanist_vols[0:2]['volume_text'].map(word_tokenize), []))
tokens.plot(30)


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Christy/nltk_data'
    - 'c:\\Users\\Christy\\miniconda3\\envs\\is310\\nltk_data'
    - 'c:\\Users\\Christy\\miniconda3\\envs\\is310\\share\\nltk_data'
    - 'c:\\Users\\Christy\\miniconda3\\envs\\is310\\lib\\nltk_data'
    - 'C:\\Users\\Christy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
